In [7]:
import pandas as pd
import re

df = pd.read_excel("..\\data\\jobdata_raw.xlsx")

# 薪资数据预处理

In [8]:
# 添加三个新列并初始化为0
df["salary_min"] = 0 # 薪资下限
df["salary_max"] = 0 # 薪资上限
df["salary_type"] = "" # 薪资类型

## 薪资数据划分
原始数据包含月薪、周薪、日薪、时薪、次结和面议等不同薪酬表达形式。为了使分析更加直观，本报告依据《中华人民共和国劳动合同法》将薪酬数据划分为**固定用工**和**灵活用工**两类，即月薪类型数据为**固定用工**，周薪、日薪、时薪、次结类型数据为**灵活用工**。
其中，固定用工与**全职**、**校招**两类招聘类型相对应，灵活用工与**实习**、**兼职**两类招聘类型相对应。

为统一薪资的表示模式，本作品通过正则表达式匹配抽取每条数据的薪资上界和下界，同时依据劳动和社会保障部颁布的《关于职工全年月平均工作时间和工资折算问题的通知》，**将不同薪酬模式均换算为年薪**以便于比较。类型为次结和面议的薪酬信息除外。

In [13]:
# 遍历每一行数据
for index, row in df.iterrows():
    salary = row['薪资']

    # 处理“面议”类型
    if salary == "面议":
        df.at[index, "salary_type"] = "N"

    # 处理“次结”类型
    match = re.match("(\d+)-(\d+)元/次", salary) # 匹配“X-Y元/次”格式
    if match: 
        df.at[index, "salary_type"] = "S"
        df.at[index, "salary_min"] = int(match.group(1))
        df.at[index, "salary_max"] = int(match.group(2))
        continue

    match = re.match("(\d+)元/次", salary) # 匹配“X元/次”格式
    if match: 
        df.at[index, "salary_type"] = "S"
        df.at[index, "salary_min"] = int(match.group(1))
        df.at[index, "salary_max"] = int(match.group(1))
        continue

    # 处理“月薪”类型
    match = re.match("(\d+(\.\d+)?)(千|万)-(\d+(\.\d+)?)(千|万)", salary) # 匹配“X(千/万)-Y(千/万)”格式
    if match:
        x_value = float(match.group(1)) * 12
        x_unit = match.group(3)
        y_value = float(match.group(4)) * 12
        y_unit = match.group(6)

        if x_unit == '千':
            x_value *= 1000
        elif x_unit == '万':
            x_value *= 10000

        if y_unit == '千':
            y_value *= 1000
        elif y_unit == '万':
            y_value *= 10000

        df.at[index, "salary_type"] = "M"
        df.at[index, "salary_min"] = int(x_value)
        df.at[index, "salary_max"] = int(y_value)
        continue

    match = re.match("(\d+(\.\d+)?)(千|万)-(\d+(\.\d+)?)(千|万)元/月", salary) # 匹配“X(千/万)-Y(千/万)元/月”格式
    if match:
        x_value = float(match.group(1)) * 12
        x_unit = match.group(3)
        y_value = float(match.group(4)) * 12
        y_unit = match.group(6)

        if x_unit == '千':
            x_value *= 1000
        elif x_unit == '万':
            x_value *= 10000

        if y_unit == '千':
            y_value *= 1000
        elif y_unit == '万':
            y_value *= 10000

        df.at[index, "salary_type"] = "M"
        df.at[index, "salary_min"] = int(x_value)
        df.at[index, "salary_max"] = int(y_value)
        continue

    match = re.match("(\d+)元/月", salary) # 匹配“X元/月”格式
    if match: 
        df.at[index, "salary_type"] = "M"
        df.at[index, "salary_min"] = int(match.group(1))*12
        df.at[index, "salary_max"] = int(match.group(1))*12
        continue

    match = re.match("(\d+(\.\d+)?)(千|万)元以下", salary) # 匹配“X(千/万)元以下”格式
    if match: 
        x_value = float(match.group(1)) * 12
        x_unit = match.group(3)

        if x_unit == '千':
            x_value *= 1000
        elif x_unit == '万':
            x_value *= 10000
        
        df.at[index, "salary_type"] = "M"
        df.at[index, "salary_min"] = 0
        df.at[index, "salary_max"] = int(x_value)
        continue


    # 处理“周薪”类型
    match = re.match("(\d+)-(\d+)元/周", salary) # 匹配“X-Y元/周”格式
    if match: 
        df.at[index, "salary_type"] = "W"
        df.at[index, "salary_min"] = int(match.group(1))*50 # 按照相关政策，周薪至年薪的换算方式为一年50周
        df.at[index, "salary_max"] = int(match.group(2))*50
        continue

    match = re.match("(\d+)元/周", salary) # 匹配“X元/周”格式
    if match: 
        df.at[index, "salary_type"] = "W"
        df.at[index, "salary_min"] = int(match.group(1))*50
        df.at[index, "salary_max"] = int(match.group(1))*50
        continue

    # 处理“日薪”类型
    match = re.match("(\d+)-(\d+)/天", salary) # 匹配“X-Y元/天”格式
    if match: 
        df.at[index, "salary_type"] = "D"
        df.at[index, "salary_min"] = int(match.group(1))*250 # 按照相关政策，日薪至年薪的换算方式为一年250天
        df.at[index, "salary_max"] = int(match.group(2))*250
        continue

    match = re.match("(\d+)元/天", salary) # 匹配“X元/天”格式
    if match: 
        df.at[index, "salary_type"] = "D"
        df.at[index, "salary_min"] = int(match.group(1))*250
        df.at[index, "salary_max"] = int(match.group(1))*250
        continue

    # 处理“时薪”类型，匹配“X-Y元/时”格式：
    match = re.match("(\d+)-(\d+)元/时", salary)
    if match: 
        df.at[index, "salary_type"] = "H"
        df.at[index, "salary_min"] = int(match.group(1))*2000 # 按照相关政策，日薪至年薪的换算方式为一年2000h
        df.at[index, "salary_max"] = int(match.group(2))*2000
        continue


# df_w_salary_type = df[df["salary_type"] == "W"]
# df_d_salary_type = df[df["salary_type"] == "D"]
# df_h_salary_type = df[df["salary_type"] == "H"]
# df_n_salary_type = df[df["salary_type"] == "N"]
# df_s_salary_type = df[df["salary_type"] == "S"]

# # print(df_w_salary_type.head(10))
# # print(df_d_salary_type.head(10))
# # print(df_h_salary_type.head(10))
# # print(df_n_salary_type.head(10))
# print(df_s_salary_type.head(10))

print(df[["salary_min", "salary_max", "salary_type"]].head(20))

    salary_min  salary_max salary_type
0        96000      144000           M
1       120000      167999           M
2        84000      120000           M
3        96000      120000           M
4       120000      144000           M
5       120000      180000           M
6        84000      167999           M
7       120000      180000           M
8       132000      240000           M
9       132000      264000           M
10      120000      180000           M
11      132000      192000           M
12      144000      180000           M
13       72000      108000           M
14      120000      204000           M
15       96000      180000           M
16      144000      180000           M
17       96000      132000           M
18       96000      144000           M
19      144000      180000           M


In [ ]:
# 删除原始“薪资”列
df = df.drop(columns = ['薪资'])

# 经验要求预处理

In [19]:
# 获取所有种类的经验要求
experiences = df['工作经验'].unique()
print(experiences)

['3-5年' '经验不限' '1-3年' '5-10年' '1年以下' '无经验' '10年以上']


In [20]:
# 合并同类值
for index, row in df.iterrows():
    exp = row['工作经验']

    if exp == '无经验': 
        df.at[index, '工作经验'] = '经验不限'
        continue

print(df['工作经验'].unique())

['3-5年' '经验不限' '1-3年' '5-10年' '1年以下' '10年以上']


In [21]:
output_file = "..\\data\\jobdata_preprocessed.xlsx"
df.to_excel(output_file, index = False, engine = 'openpyxl')